# Statsmodels

In [12]:
import pandas as pd

In [24]:
df = pd.read_csv('data/merged_data.csv')

df.drop(columns=["UK L'Oreal Paris Haircare Total Online Sellout Units",
                "UK L'Oreal Paris Haircare Online Average Price (in pound)",
                 "UK L'Oreal Paris Haircare Total Online Sellout Value (in pound)"], inplace=True)

df.rename(columns={"UK L'Oreal Paris Haircare Total Offline Sellout Units": "offline_units",
                   "UK L'Oreal Paris Haircare Offline Average Price (in pound)": "offline_average_price",
                   "UK L'Oreal Paris Haircare Total Offline Sellout Value (in pound)": "offline_sellout_value",
                   "UK L'Oreal Paris Haircare Total Weigheted Promotion Distribution (%)": "weighted_promotion_distribution"},
                    inplace=True)

In [25]:
df.growth_driver_l5.unique()

array(['influencer_management', 'bvod', 'amazon_retail', 'tesco',
       'google_video', 'youtube', 'google', 'amazon', 'citrus', 'criteo',
       'meta', 'pinterest', 'tik_tok', 'meta_collab_ads', 'the_hut_group',
       'linear', 'testers_and_merchandising'], dtype=object)

In [26]:
MMM_CATEGORIES = {
    "search": ["google", "amazon"],
    "retail_media": ["amazon_retail", "tesco", "citrus", "the_hut_group"],
    "video": ["google_video", "youtube", "bvod", "linear"],
    "social": ["meta", "pinterest", "tik_tok", "meta_collab_ads", "influencer_management"],
    "display": ["criteo"],
    "other": ["testers_and_merchandising"]
}

In [27]:
growth_columns = ["growth_driver_l1", "growth_driver_l2", "growth_driver_l3", "growth_driver_l4", "growth_driver_l5"]

df['Starting Week'] = pd.to_datetime(df['Starting Week'])

# Initialize columns to store execution and investment for each category
for category in MMM_CATEGORIES:
    df[f"{category}_impression"] = 0
    df[f"{category}_spend"] = 0

for index, row in df.iterrows():
    driver = row["growth_driver_l5"]
    for category, drivers in MMM_CATEGORIES.items():
        if driver in drivers:
            df.at[index, f"{category}_impression"] += row["execution"]
            df.at[index, f"{category}_spend"] += row["investment (in pound)"]

df = df.drop(columns=growth_columns + ["metric", "Year_x", "Year_y"])

df_grouped = df.groupby("Starting Week").agg({
    'execution': 'first',
    'offline_average_price': 'first',
    'weighted_promotion_distribution': 'first',
    'offline_sellout_value': 'first',
    'offline_units': 'first',
    'search_impression': 'sum',
    'search_spend': 'sum',
    'retail_media_impression': 'sum',
    'retail_media_spend': 'sum',
    'video_impression': 'sum',
    'video_spend': 'sum',
    'social_impression': 'sum',
    'social_spend': 'sum',
    'display_impression': 'sum',
    'display_spend': 'sum',
    'other_impression': 'sum',
    'other_spend': 'sum',

}).reset_index()

/tmp/ipykernel_6058/4108661324.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '537.2169434971748' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f"{category}_impression"] += row["execution"]
/tmp/ipykernel_6058/4108661324.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '107.43801653' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f"{category}_spend"] += row["investment (in pound)"]
/tmp/ipykernel_6058/4108661324.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '815102.08' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f"{category}_impression"] += row["execution"]
/tmp

In [28]:
import holidays

# Get UK holidays for 2022 and 2023
uk_holidays = holidays.UnitedKingdom(years=[2022, 2023])

# Convert to DataFrame for better visualization
holiday_data = pd.DataFrame(list(uk_holidays.items()), columns=["Date", "Holiday"])

# Sort the holidays by date
holiday_data['Date'] = pd.to_datetime(holiday_data['Date'])
holiday_data = holiday_data.sort_values(by='Date')

# Add a week number column
holiday_data['Week'] = holiday_data['Date'].dt.strftime('%Y-W%U')

In [29]:
# Add a column to df_grouped to indicate if there is a holiday that week
df_grouped['is_holiday'] = df_grouped['Starting Week'].dt.strftime('%Y-W%U').isin(holiday_data['Week']).astype(int)

# Display the updated DataFrame
df_grouped["is_holiday"].sum()

12

In [30]:
df_grouped

,Starting Week,execution,offline_average_price,weighted_promotion_distribution,offline_sellout_value,offline_units,search_impression,search_spend,retail_media_impression,retail_media_spend,video_impression,video_spend,social_impression,social_spend,display_impression,display_spend,other_impression,other_spend,is_holiday
0,2022-01-03,537.216943,2.323822,0.37,1583364.9,681362.5,1472980.6,4619.995672,193246.300,613.431,0.00,0.000000,2.797665e+06,3348.130667,156076.7,3053.209640,330.571429,5303.066526,1
1,2022-01-10,0.000000,2.328649,0.42,1753588.2,753049.7,2478214.7,7120.437581,343567.900,1065.064,0.00,0.000000,3.768964e+06,4242.459520,161675.8,3387.968877,261.151429,4418.402838,0
2,2022-01-17,0.000000,2.378280,0.43,1690696.8,710890.7,2695483.7,7024.529426,448357.000,1354.821,0.00,0.000000,4.650962e+06,6156.785050,408499.0,3045.849565,340.488571,6301.752894,0
3,2022-01-24,142432.121596,2.396989,0.86,1748572.8,729487.2,1944694.7,5305.028269,626307.500,2428.517,815160.71,185833.213010,1.315698e+07,49889.891558,684980.4,2990.311483,421.908314,9379.862800,0
4,2022-01-31,95869.369418,2.394615,0.78,1768055.9,738346.7,2351456.9,6059.000464,619993.400,2262.923,679352.31,305787.329227,1.708908e+07,48928.922133,1074118.5,2979.300479,513.410486,16387.142720,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-11-27,0.000000,3.067400,0.91,2472870.4,806178.1,3032230.5,11858.730000,7065943.300,38868.141,0.00,0.000000,0.000000e+00,0.000000,429973.7,8660.379000,0.000000,0.000000,0
100,2023-12-04,0.000000,3.039699,0.81,2464104.5,810641.0,1066848.9,5949.905000,9372943.450,52389.194,0.00,0.000000,0.000000e+00,0.000000,427765.0,5751.850000,0.000000,0.000000,0
101,2023-12-11,0.000000,3.057034,0.81,2686708.7,878861.1,219358.1,1986.478000,7522207.550,44554.575,0.00,0.000000,0.000000e+00,0.000000,394867.2,7118.384000,193.700000,3381.443000,0
102,2023-12-18,0.000000,3.176955,0.86,3397529.2,1069429.4,228772.7,1895.361000,5633922.450,39969.462,0.00,0.000000,0.000000e+00,0.000000,519266.8,8714.498000,0.000000,0.000000,0


In [41]:
import statsmodels.api as sm

# Define the target variable and the channels
target = 'execution'
channels = list(MMM_CATEGORIES.keys())

# Prepare the data for the model
X = df_grouped[channels]
y = df_grouped[target]

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

ModuleNotFoundError: No module named 'statsmodels'

# Meridian

In [32]:
df_grouped["geo"] = "Geo0"
df_grouped.to_csv("data/offline_binned.csv", index=False)

In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

coord_to_columns = load.CoordToColumns(
    time='Starting Week',
    geo='geo',
    controls=['weighted_promotion_distribution'],
    #population='population',
    kpi='offline_units',
    revenue_per_kpi='offline_average_price',
    media=[
        'search_impression',
        'retail_media_impression',
        'video_impression',
        'social_impression',
        'display_impression',
        'other_impression',
    ],
    media_spend=[
        'search_spend',
        'retail_media_spend',
        'video_spend',
        'social_spend',
        'display_spend',
        'other_spend',
    ],
    #organic_media=['Organic_channel0_impression'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'search_impression': 'search',
    'retail_media_impression': 'retail_media',
    'video_impression': 'video',
    'social_impression': 'social',
    'display_impression': 'display',
    'other_impression': 'other',
}

correct_media_spend_to_channel = {
    'search_spend' : "search",
    'retail_media_spend': "retail_media",
    'video_spend': "video",
    'social_spend': "social",
    'display_spend': "display",
    'other_spend': 'other',

}

loader = load.CsvDataLoader(
    csv_path="data/offline_binned.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()


2025-02-06 22:15:56.397549: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 22:15:56.407585: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 22:15:56.479086: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 22:15:56.570041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-06 22:15:56.650033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [36]:
import numpy as np

mu = 0.3
sigma = 0.8

prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(mu, sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior, knots=30 ,max_lag=8, media_effects_dist="log_normal")

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(250)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

model_diagnostics = visualizer.ModelDiagnostics(mmm)
rsquared = model_diagnostics.predictive_accuracy_table()

/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/model/model.py:67: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters 

In [37]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.predictive_accuracy_table()

/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:182: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


,metric,geo_granularity,value
0,R_Squared,national,0.807417
1,MAPE,national,0.037124
2,wMAPE,national,0.037567


In [38]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:182: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [36]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [37]:
mmm_summarizer.output_model_results_summary('summary_output.html', filepath='output')

/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/meridian/analysis/analyzer.py:182: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/charlot/Documents/work/xhec_mmm_loreal/.venv/lib/python3.

In [6]:
def durbin_watson_manual(residuals):
    numerator = np.sum(np.diff(residuals) ** 2)
    denominator = np.sum(residuals ** 2)
    return numerator / denominator

In [9]:
import json

# Load JSON data from file
with open("json_data.json", "r") as file:
    json_data = json.load(file)

# Extract dataset key dynamically
dataset_key = list(json_data.keys())[0]
data = json_data[dataset_key]

# Initialize dictionaries to store actual and expected values
actual_values = {}
expected_values = {}

# Loop through data to store values by time
for entry in data:
    time = entry["time"]
    mean_value = entry["mean"]
    
    if entry["type"] == "actual":
        actual_values[time] = mean_value
    elif entry["type"] == "expected":
        expected_values[time] = mean_value

# Compute residuals and create an array of values
residuals = np.array([
    abs(actual_values[time] - expected_values[time])
    for time in actual_values if time in expected_values
])
# Print the computed residuals
print("Residuals:", residuals)


Residuals: [4.76582500e+04 8.56587500e+04 5.13898750e+04 9.46900000e+03
 1.51473750e+04 1.90693750e+04 1.89125000e+03 4.03832500e+04
 2.40425000e+04 7.07552500e+04 1.93970000e+04 4.27026250e+04
 1.96755000e+04 9.77500000e+01 3.21852500e+04 5.62190000e+04
 5.47912500e+04 5.08076250e+04 4.73496250e+04 2.88651250e+04
 1.46406250e+04 9.17712500e+03 5.18431250e+04 4.20352500e+04
 1.41654125e+05 3.67650000e+04 7.62572500e+04 3.01260000e+04
 1.38312500e+04 8.82750000e+03 2.47775000e+03 2.25761250e+04
 4.35435000e+04 1.05912500e+04 4.66056250e+04 3.02946250e+04
 2.87742500e+04 4.62995000e+04 1.12681750e+05 5.35046250e+04
 1.68934625e+05 7.87262500e+03 8.32155000e+04 5.66868750e+04
 1.17002500e+04 2.16356250e+04 4.10221250e+04 4.16573750e+04
 3.26883750e+04 9.83125000e+03 1.88519250e+05 4.07349250e+05
 1.60705625e+05 6.97240000e+04 8.55528750e+04 7.22852500e+04
 2.09682500e+04 4.48748750e+04 3.93158750e+04 5.92565000e+04
 7.34767500e+04 6.01092500e+04 3.49382500e+04 3.52195000e+04
 1.47652250e+

In [10]:
import numpy as np

durbin_watson_manual(residuals=residuals)

0.632089830857034